# Scraping eeoh de páginas similares para comparar precios

In [2]:
#importar librarias
import requests
from lxml import html
import pandas as pd

import random
from time import sleep
from selenium import webdriver

## Scraping "holafan.com"

In [131]:
encabezados = { "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36" }
url_holafan = "https://holafan.com/collections/all"
respuesta = requests.get(url_holafan,headers=encabezados)
parser = html.fromstring(respuesta.text)

# extracción de nombres
nombre = parser.find_class('product-item__title')
nombres = [i.text_content() for i in nombre]

#extracción de precios en pesos
precio = parser.find_class('money')
precios = [i.text_content() for i in precio]

#limpieza de datos
nombre_clean = [i.strip() for i in nombres]

precio_clean = [i.strip("$, ") for i in precios]
del(precio_clean[0])

### Unir las listas "nombre" y "precios" en un dataframe

In [132]:
holafan = pd.DataFrame()
holafan["nombre"] = nombre_clean
holafan["precio"] = precio_clean
holafan["web"] = "hola fan"

In [133]:
holafan['precio'] = holafan['precio'].replace({',':''}, regex=True)

In [134]:
# ajustemos las variables numéricas
holafan['precio']=pd.to_numeric(holafan['precio'],errors='coerce')

### tipo de cambio a bs.

In [135]:
holafan['precio']=holafan['precio']*0.34

## Scraping "mocitox.com"

In [14]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://mocitox.com/famosos/')

artistas = driver.find_elements_by_xpath('//div[@class = "un-product-details"]')

In [15]:
for artista in artistas:
    nombre = artista.find_element_by_xpath('./h3/a')